<center><h1>HSE FCS SE ML</h1></center>
<center><h1>"Fast sold post prediction" Kaggle competition</h1></center>
<center><h2>Team name: Turbo 3D</h2></center>
<center><h3>Daniil Kraynov, Dmitry Strokov, Danil Kolesnikov</h3></center>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

# Read Data

In [ ]:
df = pd.read_csv('train.tsv', sep='\t').drop(columns='Unnamed: 0')

In [ ]:
df.head()

In [ ]:
df.info()

# Prepare / Encode Data

In [ ]:
# Some checks to see what columns should not be considered
print(any(df['product_id'].duplicated()))

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

def preprocess(store_data):
    lb = LabelEncoder()
    #ohe = OneHotEncoder()
    
    s = store_data.drop(columns=['product_id', 'name_text', 'desc_text', 'properties'])

    # Delivery/payment encoding: 0 is false (bad), 1 is true (good)
    s['delivery_available'] = lb.fit_transform(s['delivery_available'])
    s['payment_available'] = lb.fit_transform(s['payment_available'])
    # City/region encoding
    s['city'] = lb.fit_transform(s['city'])
    s['region'] = lb.fit_transform(s['region'])
    # Splitting date: y/m/d
    dates_split = list(map(lambda date: date.split('-'), s['date_created'].values))
    s['year_created'] = list(map(lambda split_date: int(split_date[0]), dates_split))
    s['month_created'] = list(map(lambda split_date: int(split_date[1]), dates_split))
    s['day_created'] = list(map(lambda split_date: int(split_date[2]), dates_split))
    s = s.drop(columns=['date_created'])
    # Misc
    s['owner_id'] = lb.fit_transform(s['owner_id'])
    
    # Categorical features encoding
    # Бля получается для каждой категории своя колонка будет? Все ж упадет нахуй тогда
    # categorical_cols = ['category_id', 'city', 'region', 'subcategory_id']
    # store_data = pd.get_dummies(store_data, columns=categorical_cols)
    
    return s

In [ ]:
df_p = preprocess(df)

In [ ]:
df_p.head()

In [ ]:
df_p.info()

In [ ]:
from sklearn.model_selection import train_test_split

X = df_p.drop(columns=['sold_fast'])
y = df_p['sold_fast']

print(X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,    
                                                    random_state=42)

# Some Assumptions / Graphs

In [ ]:
for feature in ('delivery_available', 'payment_available', 'img_num', 'price',
                'year_created', 'month_created'):
    print(feature, X[feature].corr(y))

# Fitting & Predicting

In [ ]:
model = Model()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Добавить тест roc-auc y_test / y_pred

# Test Data Prediction

In [ ]:
df_test = pd.read_csv('test_nolabel.tsv', sep='\t').drop(columns='Unnamed: 0')

X_actual = preprocess(df_test)

y_actual = model.predict_proba(X_actual)

df_final = pd.DataFrame.from_dict({'product_id' : df_test['product_id'].values, 'score' : y_actual})
df_final.to_csv('submission.csv', sep = ',', index = False)